<a href="https://colab.research.google.com/github/Mokshith2592/DecisionTrees/blob/main/desionTrees.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import pandas as pd
import numpy as np
from collections import Counter

In [14]:
from sklearn.model_selection import train_test_split
from sklearn import datasets

In [52]:
class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, *, value=None):
        self.feature = feature
        self.threshold = threshold
        self.value = value
        self.left = left
        self.right = right

    def is_leaf_node(self):
        return self.value is not None

In [31]:
class DecisionTree:
    def __init__(self ,min_samples_split=2 ,max_depth=100 ,n_features=None):
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.n_features = n_features
        self.root = None

    def fit(self ,X ,Y):
        self.n_features = X.shape[1] if not self.n_features else min(X.shape[1] ,self.n_features)
        self.root = self.grow_tree(X ,Y)

    def grow_tree(self ,X ,Y ,depth = 0):
        n_samples ,n_feats = X.shape
        n_labels = len(np.unique(Y))

        #check the stopping criteria
        if(depth >= self.max_depth or n_labels == 1 or n_samples < self.min_samples_split):
            leaf_val = self._most_common_label(Y)
            return Node(value = leaf_val)

        #find the best split
        feat_idxs = np.random.choice(n_feats ,self.n_features ,replace=False)
        best_feature ,best_thresh = self._best_split(X ,Y ,feat_idxs)

        #create child nodes
        left_idxs ,right_idxs = self._split(X[: ,best_feature] ,best_thresh)
        left = self.grow_tree(X[left_idxs ,:] ,Y[left_idxs] ,depth + 1)
        right = self.grow_tree(X[right_idxs ,:] ,Y[right_idxs] ,depth + 1)

        return Node(best_feature ,best_thresh ,left ,right)

    def _best_split(self ,X ,Y ,feat_idxs):
        best_gain = -1
        split_idx ,split_threshold = None ,None

        for feat_idx in feat_idxs:
            X_column = X[: ,feat_idx]
            thresholds = np.unique(X_column)

            for thr in thresholds:
                gain = self._information_gain(Y ,X_column ,thr)

                if gain > best_gain:
                    best_gain = gain
                    split_idx = feat_idx
                    split_threshold = thr

        return split_idx ,split_threshold

    def _information_gain(self ,Y ,X_column ,threshold):
        #parent Entropy
        parent_entropy = self._entropy(Y)

        #create children
        left_idx ,right_idx = self._split(X_column ,threshold)

        if len(left_idx) == 0 or len(right_idx) == 0:
            return 0

        #calculate the weighted avg entropy of children
        n = len(Y)
        n_l ,n_r = len(left_idx) ,len(right_idx)

        e_l ,e_r = self._entropy(Y[left_idx]) ,self._entropy(Y[right_idx])
        child_entropy = (n_l/n) * e_l + (n_r/n) * e_r

        #calculate the IG
        return parent_entropy - child_entropy

    def _split(self ,X_column ,threshold):
        left_idxs = np.argwhere(X_column <= threshold).flatten()
        right_idxs = np.argwhere(X_column > threshold).flatten()

        return left_idxs ,right_idxs

    def _entropy(self ,Y):
        hist = np.bincount(Y)
        ps = hist / len(Y)

        return -np.sum([p * np.log2(p) for p in ps if p > 0])

    def _most_common_label(self ,Y):
        counter = Counter(Y)
        value = counter.most_common(1)[0][0]
        return value

    def predict(self ,X):
        return np.array([self._traverse_tree(x ,self.root) for x in X])

    def _traverse_tree(self ,x ,node):
        if node.is_leaf_node():
            return node.value

        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x ,node.left)

        return self._traverse_tree(x ,node.right)

In [50]:
#testing
data = datasets.load_breast_cancer()
x ,y = data.data ,data.target

x_train ,x_test ,y_train ,y_test = train_test_split(
    x ,y ,test_size = 0.2 ,random_state = 1234
)

clf = DecisionTree(max_depth=15)
clf.fit(x_train ,y_train)

predictions = clf.predict(x_test)

In [51]:
def accuracy(y_test ,predictions):
    return np.sum(y_test == predictions) / len(y_test)

acc = accuracy(y_test ,predictions)
print(acc)

0.9298245614035088
